In [4]:
import scrapy
import scrapy.crawler as crawler
from multiprocessing import Process, Queue
from twisted.internet import reactor
import json
import time
import unidecode

import pandas as pd

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [5]:
#load romes
#Load departments

In [6]:
departments = pd.read_csv('data/department code avg.csv')
#departments = departments[1:]
departments.head()

,code,department,cp
0,1,Ain,01160
1,2,Aisne,02000
2,3,Allier,03240
3,4,Alpes-de-Haute-Provence,04000
4,5,Hautes-Alpes,05260


In [4]:
#[cp for cp in departments[departments.cp.notna()]['cp']]

In [5]:
def get_value(df, column_key, column_value, key):
    try:
        return df[df[column_key].str.startswith(key)][column_value].values[0]
    except Exception as e:
        print(e)
        return ''
    
get_value(departments, 'department', 'code', 'Paris')

'75'

In [6]:
rome_codes = pd.read_csv('data/Sncf codes romes - Sheet1.csv', names=['title', 'rome_code'])
rome_codes.head()

,title,rome_code
0,Agent technique d'unité patrimoine,C1501
1,Assistant technique Patrimoine,C1501
2,Gestionnaire technique immobilier,C1501
3,Assistant de production,C1502
4,Gestionnaire du fichier immobilier,C1502


In [7]:
base_url = 'https://labonneboite.pole-emploi.fr/'

In [8]:
options = Options()
driver = webdriver.Firefox(options=options, executable_path=r'C:/driver/geckodriver.exe')

In [9]:
driver.get(base_url)

In [10]:
test_cp_code = '02000'
test_rome_code = 'M1706'

In [11]:
search_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "j")))
search_elem.click()
search_elem.clear()
search_elem.send_keys(test_rome_code)

address_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "l")))
address_elem.click()
address_elem.clear()
address_elem.send_keys(test_cp_code)

address_elem.send_keys(Keys.ENTER)

In [18]:
driver.execute_script("window.scrollTo(0, Math.ceil(document.body.scrollHeight/5));")

#100km
distance = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "d-3")))
distance.click()


In [19]:
driver.get(str(driver.current_url)+'&from=1&to=999')

In [20]:
companies = driver.find_elements_by_class_name('lbb-bright-container')

In [21]:
results = []

for company in companies:
    try:
        h3 = company.find_element_by_tag_name('h3')
    except:
        h3 = None
        
    if h3 != None:
        #print(company.text)
        #print(company.find_element_by_class_name('rating-value').text)
        
        try:
            company_name = h3.text.split('-')[0].rstrip()
        except:
            company_name = None

        try:
            location = h3.text.split('-')[1].lstrip()
        except:
            location = None
            
        try:
            num_employees = company.find_element_by_class_name('grid-col-8').find_element_by_tag_name('p').text
        except:
            num_employees = None
            
        try:
            distance = str(company.find_element_by_class_name('travel-distance-duration').text).replace(' km de votre lieu de recherche', '')
        except:
            distance = None
        
        
        #try:
        #    distance_time = str(company.find_element_by_class_name('travel-duration').text).replace(' min en voiture', '')
        #except:
        #    distance_time = None
            
        #try:
        #    distance = distance.split(' ')[0]
        #except:
        #    pass
        
        
        
        try:
            rating = company.find_element_by_class_name('rating-value').text
        except:
            rating = None

        results.append(
            {
                'jobs': get_value(rome_codes, 'Code ROME', 'Intitulé du poste', test_rome_code), 
                'rome_code': test_rome_code, 
                'company_name': company_name,
                'location': location,
                'num_employees': num_employees,
                'distance': distance,
                #'distance_time': distance_time,
                'rating': rating,
                'cp': test_cp_code,
                'department_code': get_value(departments[departments.cp.notna()], 'code', 'department', test_cp_code)
            }
        )
    
results

'Code ROME'
index 0 is out of bounds for axis 0 with size 0


[{'jobs': '',
  'rome_code': 'M1706',
  'company_name': 'ADREXO',
  'location': 'REIMS',
  'num_employees': '100 à 199 salariés',
  'distance': '43.0',
  'rating': '3.1',
  'cp': '02000',
  'department_code': ''}]

In [16]:
len(rome_codes['rome_code'].unique()), len(departments[departments.cp.notna()]['cp'])

(52, 96)

In [11]:
departments[departments.cp.notna()][['department','code']][75:90]

,department,code
75,Paris,75
76,Seine-Maritime,76
77,Seine-et-Marne,77
78,Yvelines,78
79,Deux-Sèvres,79
80,Somme,80
81,Tarn,81
82,Tarn-et-Garonne,82
83,Var,83
84,Vaucluse,84


In [22]:
#phase2
import time

start = time.time()
print("Started")


results = []

for cp in departments[departments.cp.notna()]['cp'][75:80]:
    print('Scraping', get_value(departments[departments.cp.notna()], 'cp', 'department', cp))
    for code in rome_codes['rome_code'].unique():

        print('Scrapping: ', code)

        try:
            driver.get(base_url)

            search_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "j")))
            search_elem.click()
            search_elem.clear()
            search_elem.send_keys(code)

            address_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "l")))
            address_elem.click()
            address_elem.clear()
            address_elem.send_keys(cp)

            address_elem.send_keys(Keys.ENTER)

            driver.execute_script("window.scrollTo(0, Math.ceil(document.body.scrollHeight/5));")

            #100km
            distance = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "d-3")))
            distance.click()

            driver.get(str(driver.current_url)+'&from=1&to=999')
            companies = driver.find_elements_by_class_name('lbb-bright-container')

            for company in companies:
                try:
                    h3 = company.find_element_by_tag_name('h3')
                except:
                    h3 = None

                if h3 != None:
                    try:
                        company_name = h3.text.split('-')[0].rstrip()
                    except:
                        company_name = None

                    try:
                        location = h3.text.split('-')[1].lstrip()
                    except:
                        location = None

                    try:
                        num_employees = company.find_element_by_class_name('grid-col-8').find_element_by_tag_name('p').text
                    except:
                        num_employees = None

                    try:
                        distance = str(company.find_element_by_class_name('travel-distance-duration').text).replace(' km de votre lieu de recherche', '')
                    except:
                        distance = None
                    try:
                        rating = company.find_element_by_class_name('rating-value').text
                    except:
                        rating = None

                    results.append(
                        {
                            'jobs': get_value(rome_codes, 'rome_code', 'title', code), 
                            'rome_code': code, 
                            'company_name': company_name,
                            'location': location,
                            'num_employees': num_employees,
                            'distance': distance,
                            #'distance_time': distance_time,
                            'rating': rating,
                            'cp': cp,
                            'department': get_value(departments[departments.cp.notna()], 'cp', 'department', cp)
                        }
                    )
        except:
            pass

        time.sleep(2)
    
results

end = time.time()
print("Finished", end - start)

Started
Scraping Paris
Scrapping:  C1501
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vecto

cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index

cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index

cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index

cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index

...

cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
Scrapping:  M1202
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN v

cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
Scrapping:  M1205
cannot index with vector containing NA / NaN v

cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
Scrapping:  M1503
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN v

cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index

cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index with vector containing NA / NaN values
cannot index

In [23]:
pd.DataFrame(results).to_csv('data/labonneboite_66_70.csv', index=False)

In [24]:
pd.DataFrame(results).to_clipboard(index=False, header=False)  

In [12]:
#pd.DataFrame(results).to_csv('data/labonneboite_phase2.csv', index=False)
pd.DataFrame(results, header=None)

NameError: name 'results' is not defined

In [151]:
#phase1
'''
import time

results = []

for code in rome_codes['Code ROME']:
    
    print('Scrapping: ', code)
    
    try:

        driver.get(base_url)

        search_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "j")))
        search_elem.click()
        search_elem.clear()
        search_elem.send_keys(code)

        address_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "l")))
        address_elem.click()
        address_elem.clear()
        address_elem.send_keys("Villecomtal-sur-Arros 32730")

        address_elem.send_keys(Keys.ENTER)

        driver.execute_script("window.scrollTo(0, Math.ceil(document.body.scrollHeight/5));")

        #100km
        distance = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "d-4")))
        distance.click()

        driver.get(str(driver.current_url)+'&from=1&to=999')
        companies = driver.find_elements_by_class_name('lbb-bright-container')

        for company in companies:
            try:
                h3 = company.find_element_by_tag_name('h3')
            except:
                h3 = None

            if h3 != None:
                #print(company.text)
                #print(company.find_element_by_class_name('rating-value').text)

                try:
                    company_name = h3.text.split('-')[0].rstrip()
                except:
                    company_name = None

                try:
                    location = h3.text.split('-')[1].lstrip()
                except:
                    location = None

                try:
                    num_employees = company.find_element_by_class_name('grid-col-8').find_element_by_tag_name('p').text
                except:
                    num_employees = None

                try:
                    distance = str(company.find_element_by_class_name('travel-distance-duration').text).replace(' km de votre lieu de recherche', '')
                except:
                    distance = None

                
                #try:
                #    distance_time = str(company.find_element_by_class_name('travel-duration').text).replace(' min en voiture', '')
                #except:
                #    distance_time = None

                #try:
                #    distance = distance.split(' ')[0]
                #except:
                #    pass
                

                try:
                    rating = company.find_element_by_class_name('rating-value').text
                except:
                    rating = None

                results.append(
                    {
                        'jobs': rome_codes[rome_codes['Code ROME'].str.startswith(code)]['Intitulé du poste '].values[0], 
                        'rome_code': code, 
                        'company_name': company_name,
                        'location': location,
                        'num_employees': num_employees,
                        'distance': distance,
                        #'distance_time': distance_time,
                        'rating': rating
                    }
                )
    except:
        pass
    
    time.sleep(10)
    
results
'''

'\nimport time\n\nresults = []\n\nfor code in rome_codes[\'Code ROME\']:\n    \n    print(\'Scrapping: \', code)\n    \n    try:\n\n        driver.get(base_url)\n\n        search_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "j")))\n        search_elem.click()\n        search_elem.clear()\n        search_elem.send_keys(code)\n\n        address_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "l")))\n        address_elem.click()\n        address_elem.clear()\n        address_elem.send_keys("Villecomtal-sur-Arros 32730")\n\n        address_elem.send_keys(Keys.ENTER)\n\n        driver.execute_script("window.scrollTo(0, Math.ceil(document.body.scrollHeight/5));")\n\n        #100km\n        distance = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "d-4")))\n        distance.click()\n\n        driver.get(str(driver.current_url)+\'&from=1&to=999\')\n        companies = driver.find_elements_by_class_name(\'lbb

In [136]:
df = pd.read_csv('data/labonneboite.csv')
df

,company_name,distance,jobs,location,num_employees,rating,rome_code
0,AXE TRAVAIL TEMPORAIRE,42.0 48 min en voiture,Adjointe RH,AUCH,1 ou 2 salariés,2.5,M1501
1,O2,78.6 89 min en voiture,Adjointe RH,MONT,0 salarié,2.5,M1501
2,MAIRIE,100.0 105 min en voiture,Adjointe RH,BLAGNAC,0 salarié,4.2,M1501
3,MANPOWER FRANCE,93.5 112 min en voiture,Adjointe RH,BOE,3 à 5 salariés,4.4,M1501
4,SDIS64,45.1 57 min en voiture,Adjointe RH,PAU,0 salarié,4.4,M1501
5,RANDSTAD,91.4 91 min en voiture,Adjointe RH,MURET,0 salarié,4.5,M1501
6,AQUILA RH,47.4 58 min en voiture,Adjointe RH,SERRES,1 ou 2 salariés,3.8,M1501
7,MAIRIE,47.9 59 min en voiture,Adjointe RH,PAU,1 000 à 1 999 salariés,3.9,M1501
8,LE&C GS,97.2 101 min en voiture,Adjointe RH,TOULOUSE,1 000 à 1 999 salariés,4.4,M1501
9,CA PAU BEARN PYRENEES,47.8 59 min en voiture,Adjointe RH,PAU,0 salarié,4.2,M1501


In [137]:
df['distance'][1].split(' ')[0]

'78.6'

In [138]:
'98.2'.split(' ')[0]

'98.2'

In [139]:
df['distance'] = [distance.split(' ')[0] for distance in df['distance']]

In [130]:
df.to_csv('data/labonneboite_parsed.csv', index=False)

In [142]:
df.to_clipboard()

In [140]:
df['num_employees'] = [num_employees.replace('1 000', '1000').replace('10 000', '10000').replace('2 000', '2000').replace('5 000', '5000').split(' ')[0] for num_employees in df['num_employees']]

In [141]:
df

,company_name,distance,jobs,location,num_employees,rating,rome_code
0,AXE TRAVAIL TEMPORAIRE,42.0,Adjointe RH,AUCH,1,2.5,M1501
1,O2,78.6,Adjointe RH,MONT,0,2.5,M1501
2,MAIRIE,100.0,Adjointe RH,BLAGNAC,0,4.2,M1501
3,MANPOWER FRANCE,93.5,Adjointe RH,BOE,3,4.4,M1501
4,SDIS64,45.1,Adjointe RH,PAU,0,4.4,M1501
5,RANDSTAD,91.4,Adjointe RH,MURET,0,4.5,M1501
6,AQUILA RH,47.4,Adjointe RH,SERRES,1,3.8,M1501
7,MAIRIE,47.9,Adjointe RH,PAU,1000,3.9,M1501
8,LE&C GS,97.2,Adjointe RH,TOULOUSE,1000,4.4,M1501
9,CA PAU BEARN PYRENEES,47.8,Adjointe RH,PAU,0,4.2,M1501
